# SQL Queries on CMS' 2010 Medicare Beneficiary Data

This project uses Python's Postgres client, _psycopg2_ to carry out SQL queries on a smaller set of CMS' synthetic 2010 Medicare Beneficiary data.

## Python's psycopg2

The documentation for psycopg2 can be found online. If you do not already have psycopg2 you can pip install psycopg2 (if you see errors later in this page it is not installed). First, I will import the package:

In [2]:
import psycopg2

To use psycopg2, I first connected to a running Postgres database using the database name, my username, a host IP or address or localhost if running the database locally, a port number, and possibly a password. These parameters are set below. Adjust the dbname and user (and password if needed) to match where you stored your CMS data.

In [3]:
dbname='alysonlentz'
user='alysonlentz'
password=''

To create a nice looking tabular output from the SQL queries, I utilized the _pandas_ Python package. I created a function that accepts the SQL statement, creates its own cursor, executes the SQL query, and then prints the results in a nice tabular format.

In [6]:
import pandas as pd
def tabular(SQL):
    """Displays the results of a Postgres query using psycopg2 in table form.
    
    Parameters
    ----------
    SQL : str
        A valid SQL query.
    """
    pd.set_option('display.max_rows', 50)
    con = psycopg2.connect(dbname=dbname, user=user, host='localhost', port=5432, password=password)
    cur = con.cursor()
    try:
        cur.execute(SQL)
    except (psycopg2.Error) as e:
        con.close()
        print 'Invalid query!'
    colnames = [desc[0] for desc in cur.description]
    df = pd.DataFrame(cur.fetchall(), columns=colnames)
    print df

## Example 1:

Find the average months of HMO coverage when the patient was reported to have cancer.

In [12]:
SQL = """
SELECT AVG(hmo_mo) AS avg_hmo_mo
FROM
    (SELECT id FROM cmspop WHERE cancer = True) AS LHS
LEFT JOIN
    (SELECT id, hmo_mo FROM cmsclaims) AS RHS
ON LHS.id = RHS.id;
"""
tabular(SQL)

           avg_hmo_mo
0  3.7023121636911702


## Example 2:

The claims where id contains either “000” or “34”.

In [15]:
SQL = """
SELECT * 
FROM 
  cmspop 
WHERE id LIKE '%000%' OR id LIKE '%34%'
LIMIT 5;
"""
tabular(SQL)

                 id         dob   dod     sex      race state  county  \
0  00013D2EFD8E45D1  1923-05-01  None    male     white    MO     950   
1  00016F745862898F  1943-01-01  None    male     white    PA     230   
2  0001FDD721E223DC  1936-09-01  None  female     white    PA     280   
3  00021CA6FF03E670  1941-06-01  None    male  hispanic    CO     290   
4  00024B3D2352D2D0  1936-08-01  None    male     white    WI     590   

  alz_rel_sen heart_fail cancer depression  
0       False       True  False      False  
1        True       True  False      False  
2       False      False  False      False  
3       False      False  False      False  
4       False      False  False      False  


## Example 3:

Find the total carrier reimbursements for every state, ordered by state in ascending order.

In [16]:
SQL = """
SELECT state, SUM(RHS.carrier_reimb) AS total_carrier_reimb
FROM
    (SELECT id, state FROM cmspop) AS LHS
LEFT JOIN
    (SELECT id, carrier_reimb FROM cmsclaims) AS RHS
ON LHS.id = RHS.id
GROUP BY state
ORDER BY state ASC;
"""
tabular(SQL)

   state  total_carrier_reimb
0     AK              2772920
1     AL             37954040
2     AR             24332640
3     AZ             32223660
4     CA            154337620
5     CO             23097780
6     CT             24599990
7     DC              3312650
8     DE              7357400
9     FL            139028200
10    GA             54178430
11    HI              6486680
12    IA             23215920
13    ID              9103040
14    IL             81784840
15    IN             44170960
16    KS             20141320
17    KY             33882710
18    LA             28909950
19    MA             44394180
20    MD             36109930
21    ME             11912230
22    MI             71438500
23    MN             29637730
24    MO             42094250
..   ...                  ...
27    NC             65286040
28    ND              5406490
29    NE             12529690
30    NH             10448720
31    NJ             60752050
32    NM             12132570
33    NV  

## Example 4:

Calculate the ratio of carrier reimbursement to beneficiary responsibility in descending order.

In [18]:
SQL = """
SELECT (carrier_reimb/bene_resp)::float AS ratio 
FROM 
  cmsclaims 
WHERE bene_resp <> 0 
ORDER BY ratio DESC
LIMIT 10;
"""
tabular(SQL)

   ratio
0  155.0
1  127.0
2  116.0
3  115.0
4  109.0
5  109.0
6   95.0
7   93.0
8   89.0
9   87.0


## Example 5:

Find out which state spends the most money on carrier reimbursements for depression. Return columns for the state and total carrier reimbursements spent on depression.

In [20]:
SQL = """
SELECT state, SUM(RHS.carrier_reimb) AS total_carrier_reimb_on_depression
FROM
    (SELECT id, state FROM cmspop WHERE depression = True) AS LHS
LEFT JOIN
    (SELECT id, carrier_reimb FROM cmsclaims) AS RHS
ON LHS.id = RHS.id
GROUP BY state
ORDER BY total_carrier_reimb_on_depression DESC
LIMIT 1;
"""
tabular(SQL)

  state  total_carrier_reimb_on_depression
0    CA                           49975370


## Example 6:

For every value of carrier reimbursement, how many total months of HMO coverage were provided? Order by carrier reimbursements in descending order.

In [22]:
SQL = """
SELECT carrier_reimb, SUM(hmo_mo) AS total_hmo_months 
FROM 
  cmsclaims 
GROUP BY carrier_reimb 
ORDER BY carrier_reimb DESC;
"""
tabular(SQL)

     carrier_reimb  total_hmo_months
0            16530                12
1            13040                 0
2            12500                12
3            11730                 0
4            11630                 0
5            11380                 0
6            11340                12
7            11210                 5
8            11100                 0
9            11050                 0
10           10930                 0
11           10920                12
12           10890                 7
13           10830                12
14           10790                 0
15           10760                 0
16           10730                 5
17           10700                12
18           10610                12
19           10490                 0
20           10420                12
21           10400                12
22           10390                12
23           10350                12
24           10310                12
..             ...               ...
9

## Example 7:

Rank each state by their number of heart failure claims in proportion to their total claims, ordered by the proportion in descending order. Your query should return two columns.

In [23]:
SQL = """
SELECT LHS.state, (LHS.num_heart_fail_claims::float / RHS.num_claims) AS prop_hf_claims
FROM 
  (SELECT state, COUNT(*) AS num_heart_fail_claims FROM cmspop WHERE heart_fail = True GROUP BY state) AS LHS
LEFT JOIN 
  (SELECT state, COUNT(*) AS num_claims FROM cmspop GROUP BY state) AS RHS
ON LHS.state = RHS.state 
ORDER BY prop_hf_claims DESC;
"""
tabular(SQL)

   state  prop_hf_claims
0     NJ        0.299248
1     IL        0.299172
2     MD        0.295338
3     TX        0.295151
4     KY        0.295104
5     SC        0.288719
6     IN        0.285921
7     KS        0.282834
8     MI        0.282747
9     MS        0.280962
10    OH        0.280505
11    MO        0.280107
12    NY        0.279261
13    FL        0.276832
14    IA        0.276802
15    VA        0.275898
16    GA        0.273272
17    LA        0.271915
18    NE        0.271429
19    MA        0.269978
20    ME        0.268407
21    NC        0.262323
22    TN        0.260019
23    PA        0.257831
24    CT        0.255286
..   ...             ...
27    MN        0.246830
28    OK        0.246060
29    CA        0.241850
30    SD        0.240586
31    MT        0.235521
32    WI        0.233023
33    AL        0.232198
34    WV        0.231978
35    NV        0.231320
36    DE        0.230977
37    VT        0.229393
38    AZ        0.218221
39    AR        0.209706


## Example 8:

Return the state and number of heart failures for the state with the greatest number of heart failures.

In [24]:
SQL = """
SELECT state, COUNT(heart_fail) AS num_heart_failures 
FROM 
  cmspop 
WHERE heart_fail = True 
GROUP BY state
ORDER BY num_heart_failures DESC 
LIMIT 1;
"""
tabular(SQL)

  state  num_heart_failures
0    CA               47281


## Example 9:

Return the id, sex, race, and age in integer years at the time of data collection for the oldest living person in Massachusetts that has all complications (Alzheimers, depression, cancer, and heart failure).

In [31]:
SQL = """
SELECT id, sex, race, FLOOR(2010 - CAST(SUBSTR(dob, 1, 4) AS int)) AS age 
FROM
  cmspop 
WHERE alz_rel_sen = 't' AND heart_fail = 't' AND cancer = 't' AND depression
= 't' AND state = 'MA' AND dod IS NULL 
ORDER BY age DESC 
LIMIT 1;
"""
tabular(SQL)

                 id     sex   race    age
0  AB33557A62C914C7  female  white  101.0


## Example 10:

Imagine you are doing a social study on the health of certain races in different regions of the country. For the race that most frequently submits claims in Texas, find the state which has the lowest frequency of claims from the same race. Return the percentage of carrier reimbursement cost that race is responsible for, the average number of HMO months of coverage, and average beneficiary responsibility for those two states and that race. Order by state in ascending order.

In [33]:
SQL = """
SELECT state, race,
       ((total_reimb / aggregate_carrier_reimb::float) * 100)::float AS percent_of_reimbs,
       avg_hmo_mo, avg_bene_resp
FROM
   (SELECT LLHS.state, LLHS.race, LLHS.total_carrier_reimb AS total_reimb, LLHS.avg_hmo_mo,
           LLHS.avg_bene_resp, RRHS.aggregate_carrier_reimb
    FROM
        (SELECT LHS.state, LHS.race, SUM(carrier_reimb) AS total_carrier_reimb,
                AVG(hmo_mo) AS avg_hmo_mo, AVG(bene_resp) AS avg_bene_resp FROM
           (SELECT id, state, race
            FROM cmspop
            WHERE state IN
              (SELECT state
               FROM
                 /* OUR PREVIOUS QUERY */
                 (SELECT state, COUNT(*) AS num_claims_single_race
                  FROM cmspop
                  WHERE race IN 
                    (SELECT race
                     FROM
                      (SELECT race, COUNT(*) AS num_claims
                       FROM cmspop
                       WHERE state = 'TX'
                       GROUP BY race
                       ORDER BY num_claims DESC
                       LIMIT 1
                      ) AS sub_sub_q
                     )
                  GROUP BY state
                  ORDER BY num_claims_single_race ASC
                  LIMIT 1) AS sub_q
                 /* END PREVIOUS QUERY */
              )
              OR state = 'TX') AS LHS
        LEFT JOIN
           (SELECT id, carrier_reimb, hmo_mo, bene_resp FROM cmsclaims) AS RHS
        ON LHS.id=RHS.id
        GROUP BY state, race) AS LLHS
    LEFT JOIN
        (SELECT state, SUM(carrier_reimb) AS aggregate_carrier_reimb FROM
           (SELECT id, state, race
            FROM cmspop
            WHERE state IN
              (SELECT state
               FROM
                 /* OUR PREVIOUS QUERY */
                 (SELECT state, COUNT(*) AS num_claims_single_race
                  FROM cmspop
                  WHERE race IN 
                    (SELECT race
                     FROM
                      (SELECT race, COUNT(*) AS num_claims
                       FROM cmspop
                       WHERE state = 'TX'
                       GROUP BY race
                       ORDER BY num_claims DESC
                       LIMIT 1
                      ) AS sub_sub_q
                     )
                  GROUP BY state
                  ORDER BY num_claims_single_race ASC
                  LIMIT 1) AS sub_q
                 /* END PREVIOUS QUERY */
              )
              OR state = 'TX') AS LHS
        LEFT JOIN
           (SELECT id, carrier_reimb, hmo_mo, bene_resp FROM cmsclaims) AS RHS
        ON LHS.id=RHS.id
        GROUP BY state) AS RRHS
    ON LLHS.state = RRHS.state) AS large_outer_q
WHERE race IN
  (SELECT race
   FROM
     (SELECT race, COUNT(*) AS num_claims
      FROM cmspop
      WHERE state = 'TX'
      GROUP BY race
      ORDER BY num_claims DESC
      LIMIT 1
     ) AS sub_q
  )
ORDER BY state ASC;
"""
tabular(SQL)

  state   race  percent_of_reimbs          avg_hmo_mo         avg_bene_resp
0    DC  white          37.944546  1.7576295244854507  126.2242725337118524
1    TX  white          82.183373  3.7853097482446953  284.4598316459133403
